# Human Mobility in San Diego County
Jessica Embury

### Drop tables from the database as needed

In [3]:
#import needed libraries
from bs4 import BeautifulSoup
import requests
import os
import glob
import gzip
import psycopg2

In [10]:
#USER ENTERED VARIABLES

#directory where this file is located and where data files will be stored
abs_path = ''

#for database 
host = ''
db = ''
user = ''
password = ''

schema_name = 'lodes'

#For fresh start (get all files for all states), use state_num = 0 and file_num = 0
#Change state_num and file_num if data retrieval needs to be resumed (after SSL Error, etc.)

#Enter CURRENT state number
#'az' = 0, 'ca' = 1, 'co' = 2, 'dc' = 3, 'fl' = 4, 'il' = 5, 'ma' = 6, 'mi'  = 7, 
#'mn' = 8, 'nv' = 9, 'ny' = 10, 'tx' = 11, 'wa' = 12
state_num = 0

#Enter last SUCCESSFUL file number for the current state (For example, in 'az', 71 out of 136, first file will be 72)
file_num = 0

In [14]:
#list object containing states to get lodes data from
#states = ['az', 'ca', 'co', 'dc', 'fl', 'il', 'ma', 'mi', 'mn', 'nv', 'ny', 'tx', 'wa']
states = []

In [7]:
#FUNCTIONS

#Function to get all file names from the associated url 
#reference: https://stackoverflow.com/questions/11023530/python-to-list-http-files-and-directories
def listFD(url, ext=''):
    page = requests.get(url).text
    #print(page)
    soup = BeautifulSoup(page, 'html.parser')
    return [url + '/' + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]

#function for psycopg2 to connect to the PostgreSQL database server
#reference: https://github.com/NaysanSaran/pandas2postgresql/blob/master/notebooks/CompleteExample.ipynb
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    return conn

In [12]:
#CONNECT TO DATABASE

#specify the psql connection parameters
param_dic = {
    "host"      : host,
    "database"  : db,
    "user"      : user,
    "password"  : password,
}

#connect to the database server
conn = connect(param_dic)

#define the cursor
cur = conn.cursor()

Connecting to the PostgreSQL database...


In [15]:
#delete all tables from database for states in list
for y in range(len(states) - state_num):
    
    #adjust to current state
    i = y + state_num
    
    #list object to contain names of all files to be downloaded from url - for current state
    filenames = []
    
    #variable for url to the current state's lodes files
    url = 'https://lehd.ces.census.gov/data/lodes/LODES7/' + states[i] + '/od/'
    
    #file extension of lodes data
    ext = 'csv.gz'

    #get all file names for the current state
    for file in listFD(url, ext):
        #print(file)
        filenames.append(os.path.basename(file))
    
    print('State: ' +states[i] + ', Number of files for ' + url +' = ' + str(len(filenames)))
    
        
    #for each file for the current state:
    for z in range(len(filenames) - file_num):
            
        #adjust to next file
        file = z + file_num
      
        #DATABASE SET UP
        
        #database table name - same as filename, but remove extensions (.csv.gz)
        table_name = schema_name + '.' + filenames[file][:-7]
        
        #drop table if exists
        drop_table = ('DROP TABLE IF EXISTS {};'.format(table_name))

        try:
            cur.execute(drop_table)
            print('Table dropped if exists.')
        except:
            print("Error. Table (if exists) not dropped.")
            
    #commit database changes for each state
    conn.commit()

State: dc, Number of files for https://lehd.ces.census.gov/data/lodes/LODES7/dc/od/ = 88
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropped if exists.
Table dropp

In [16]:
#COMMIT AND CLOSE DATABASE
#commit changes to the database
conn.commit()

#close the database connection
conn.close()

#close the cursor
cur.close()